In [1]:
import numpy as np
import mlx.core as mx
import time
import cv2
import os

In [2]:
rng = np.random.default_rng(1)

In [3]:
folder = "./data/"

img_files = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

bases = []
actives = []

for _file in img_files:
    img = cv2.imread(_file, cv2.IMREAD_GRAYSCALE)
    rnd = (rng.random(img.shape, dtype = np.float32) * 255).astype(np.uint8)
    bases.append(img)
    actives.append(rnd)

In [4]:
mx.random.seed(0)

def mat_runner(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        b = mx.array(bases[i], mx.uint8)
        a = mx.array(actives[i], mx.uint8)
        start_time = time.perf_counter()
        mx.eval(f(b, a))
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def mat_runner_float(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        base = bases[i].astype(np.float32)
        active = actives[i].astype(np.float32)
        b = mx.array(base, mx.float32)
        a = mx.array(active, mx.float32)
        opacity = mx.array(rng.random(1, dtype = np.float32), mx.float32)
        start_time = time.perf_counter()
        mx.eval(f(b, a, opacity))
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def vec_runner_int(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        base = bases[i].flatten()
        active = actives[i].flatten()
        b = mx.array(base, mx.uint8)
        a = mx.array(active, mx.uint8)
        opacity = mx.array(rng.random(1, dtype = np.float32).astype(np.uint8), mx.uint8)
        start_time = time.perf_counter()
        mx.eval(f(b, a, opacity))
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def vec_runner_float(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        base = bases[i].flatten().astype(np.float32)
        active = actives[i].flatten().astype(np.float32)
        b = mx.array(base, mx.float32)
        a = mx.array(active, mx.float32)
        opacity = mx.array(rng.random(1, dtype = np.float32), mx.float32)
        start_time = time.perf_counter()
        mx.eval(f(b, a, opacity))
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def timer(input1, input2, f, runner):
    runs = 10
    times = []
    for _ in range(runs):
        times.append(runner(input1, input2, f))
    times = np.array(times)
    print(f"{f.__name__}")
    print(f"{np.average(times)}ms +/- {np.std(times)}ms")

In [5]:
###MLX

In [ ]:
def dissolve_blend_8_mx(base, active, opacity):
    return mx.where(mx.greater_equal(opacity - ((mx.random.randint(1, 2147483647, base.shape, mx.int32) % 100) + 1) / 100, 0), active, base)

In [6]:
def darken_blend_8_mx(base, active):
  return mx.where(mx.greater(base, active), active, base)

In [7]:
def color_burn_8_mx(base, active):
  return mx.where(mx.equal(active, 0), 255, (255) - (((255) - (base)) // (active)))

In [8]:
def lighten_blend_8_mx(base, active):
  return mx.where(mx.less(base, active), active, base)

In [9]:
def color_dodge_8_mx(base, active):
  return mx.where(mx.equal(active, 255), 255, (base) // ((255) - (active)))

In [10]:
def overlay_blend_8_mx(base, active):
  return mx.where(mx.greater_equal(base, 128), ((((2) * (base)) + (base)) - ((((2) * (base)) * (base)) // (255))) - (255), (((2) * (base)) * (base)) // (255))

In [11]:
def multiply_blend_8_mx(base, active):
  return ((base) * (active)) // (255)

In [12]:
def linear_burn_8_mx(base, active):
  return ((base) + (active)) - (255)

In [13]:
def screen_blend_8_mx(base, active):
  return ((base) + (active)) - (((base) * (active)) // (255))

In [14]:
def linear_dodge_8_mx(base, active):
  return (base) + (active)

In [16]:
def normal_blend_f_mx(base, active, opacity):
  return ((opacity) * (active)) + (((1) - (opacity)) * (base))

In [17]:
def normal_blend_8_mx(base, active, opacity):
  return ((opacity) * (active)) + (((255) - (opacity)) * (base))

In [18]:
timer(bases, actives, darken_blend_8_mx, mat_runner)

darken_blend_8_mx
2732.5353465033118ms +/- 111.77568220952709ms


In [19]:
timer(bases, actives, color_burn_8_mx, mat_runner)

color_burn_8_mx
3095.4330299984917ms +/- 6.53067596107768ms


In [20]:
timer(bases, actives, lighten_blend_8_mx, mat_runner)

lighten_blend_8_mx
2850.779464501045ms +/- 30.118045608143987ms


In [21]:
timer(bases, actives, color_dodge_8_mx, mat_runner)

color_dodge_8_mx
3093.87934450084ms +/- 17.12700450464597ms


In [22]:
timer(bases, actives, overlay_blend_8_mx, mat_runner)

overlay_blend_8_mx
3726.584408800022ms +/- 467.4370542287949ms


In [23]:
timer(bases, actives, multiply_blend_8_mx, mat_runner)

multiply_blend_8_mx
2253.984360094773ms +/- 11.784597429968985ms


In [24]:
timer(bases, actives, linear_burn_8_mx, mat_runner)

linear_burn_8_mx
2232.258692804203ms +/- 5.284886579118435ms


In [25]:
timer(bases, actives, screen_blend_8_mx, mat_runner)

screen_blend_8_mx
2507.485742397512ms +/- 9.541389124696947ms


In [26]:
timer(bases, actives, linear_dodge_8_mx, mat_runner)

linear_dodge_8_mx
1697.4647873988033ms +/- 49.79813995317829ms


In [27]:
timer(bases, actives, normal_blend_f_mx, vec_runner_float)

normal_blend_f_mx
5044.476787297912ms +/- 350.6389352462887ms


In [28]:
timer(bases, actives, normal_blend_8_mx, vec_runner_int)

normal_blend_8_mx
3394.0634279995493ms +/- 133.99528305213315ms


In [31]:
timer(bases, actives, dissolve_blend_8_mx, mat_runner_float)

dissolve_blend_8_mx
18712.383793596753ms +/- 782.7154854909116ms
